# An example of parameters estimation using supernovas


This is just a fancy way to visualize data 

In [1]:
def PrettyPrint(data):
    from prettytable import PrettyTable
    x = PrettyTable(data.dtype.names)    
    for row in data:
        x.add_row(row)
    print(x)

Luminosity distance is defined in  terms of an integral that depends on the cosmological parameters that want to be estimated. If we define the function $$E(z) = \sqrt{(1-\Omega_{m0}) + \Omega_{m0}(1+z)^{3} + \Omega_{R0}(1+z)^{4} },$$ the luminosity distance is given by: $$d_{L} = \frac{3\times 10^{3}}{h}(1+z)\int_{0}^{z}\frac{dz'}{E(z')}$$

In [2]:
def Integrand(z, Omegas):
    import numpy as np
    E =  np.sqrt( (1 - Omegas[0]) + Omegas[0] * np.power(1 + z, 3) + Omegas[1] * np.power(1 + z, 4 ) )
    return 1. / E

In [3]:
def dl(z, h, Omegas):
    import numpy as np
    from scipy.integrate import quad
    I,e = quad(Integrand, 0, z,  args=(Omegas))
    return 3000 * (1 + z) * I / h

The module distance is the quantity that is related with the experimental data, because we do not measure distance directly. From the definitions of apparent and absolute magnitude  we have. $$\mu = m-M = 5\log{d_{L}}+25$$

In [4]:
def mu(dl):
    import numpy as np
    return 5 * np.log10(dl) + 25

And we define our $\chi^{2}$ test respect to this quantity. So we want to find the parameters that minimizes 
$$\chi^{2}= \sum \left(\frac{\mu_{teo}-\mu_{exp}}{\sigma_{mu}}\right)^{2} $$ 

In [5]:
def chi2(mut, muobs,  sigma):
    import numpy as np
    aux = 0
    for i in range(len(mut)):
        #aux += np.power( (mut[i] - muobs[i]) / sigma[i], 2)
        aux += np.power((mut[i] - muobs[i]), 2)/( np.power( sigma[i], 2) + 0.0169)
    return aux

This function performs a run with two free parameters: $h$ and $\Omega_{m0}$. It is not optimized, so it will run over a full set of values defined uniformly for each parameter. The first three args correspond to the number of divisions, the min value and the maximum value for the range of the $h$ parameter. In the same way are defined the following three arguments.

In [6]:
def runhOm(steps1, minval1,  maxval1, steps2, minval2, maxval2, data):
    import numpy as np
    import itertools as it
    import pandas as pd
    muobs = data['m-M']
    sigma =  data['sigma']
    muteo = [None] * len(muobs)
    h = 0
    Omegas =  [None,None]
    Omegas[1] = 0.000084
    
    #Output data
    keys =  ["h", "Om",  "Or",  "chi2"]
    outdata = { key : [None] for key in keys }  
    delta1 =  (maxval1 - minval1) / steps1
    delta2 =  (maxval2 - minval2) / steps2
    for i, j in it.product(range(steps1),  range(steps2)) :
        h = minval1 + delta1* i
        Omegas[0] = minval2 + delta2*j
        for k in range(len(data['z'])):
            Z =  data['z'][k]
            muteo[k] =  mu( dl(Z ,  h,  Omegas) )
        chiaux = chi2(muteo, muobs, sigma)
        outdata["h"].append(h)
        outdata["Om"].append(Omegas[0])
        outdata["Or"].append(Omegas[1])
        outdata["chi2"].append(chiaux)
        #print (Omegas[0] ," ",Omegas[1] ," ",h , " ", chiaux)

    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('h =', list(df.loc[boolean, 'h'])[0] ,  '\n' )
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Omega_r =', list(df.loc[boolean, 'Or'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reducedchi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 2) ,  '\n' )
 

The following two function performs the run assuming that one of the parameters is well known, whose value is the last argument

In [7]:
def runOm(steps, minval, maxval,  data,  h):
    import numpy as np
    import itertools as it
    import pandas as pd
    muobs = data['m-M']
    sigma =  data['sigma']
    muteo = [None] * len(muobs)
    Omegas =  [None,None]
    Omegas[1] = 0.000084
    
    #Output data
    keys =  ["h", "Om",  "Or",  "chi2"]
    outdata = { key : [None] for key in keys }  
    delta =  (maxval - minval) / steps
    
    for j in range(1, steps + 1):
        Omegas[0] = minval + delta * j
        for k in range(len(data['z'])):
            Z =  data['z'][k]
            muteo[k] =  mu( dl(Z ,  h,  Omegas) )
        chiaux = chi2(muteo, muobs, sigma)
        outdata["h"].append(h)
        outdata["Om"].append(Omegas[0])
        outdata["Or"].append(Omegas[1])
        outdata["chi2"].append(chiaux)
        #print (Omegas[0] ," ",Omegas[1] ," ",chiaux , " ", h)

    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('h =', list(df.loc[boolean, 'h'])[0] ,  '\n' )
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Omega_r =', list(df.loc[boolean, 'Or'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reduced_chi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 1) ,  '\n' )

In [8]:
def runh(steps, minval, maxval,  data,  Om):
    import numpy as np
    import itertools as it
    import pandas as pd
    muobs = data['m-M']
    sigma =  data['sigma']
    muteo = [None] * len(muobs)
    Omegas =  [None,None]
    Omegas[1] = 0.000084
    Omegas[0] =  Om  
    #Output data
    keys =  ["h", "Om",  "Or",  "chi2"]
    outdata = { key : [None] for key in keys }
    delta =  (maxval - minval) / steps
    for i in range(1, steps + 1):
        h = minval + delta * i     
        for k in range(len(data['z'])):
            Z =  data['z'][k]
            muteo[k] =  mu( dl(Z ,  h,  Omegas) )
        chiaux = chi2(muteo, muobs, sigma)
        outdata["h"].append(h)
        outdata["Om"].append(Omegas[0])
        outdata["Or"].append(Omegas[1])
        outdata["chi2"].append(chiaux)
        #print (Omegas[0] ," ",Omegas[1] ," ",chiaux , " ", h)

    df = pd.DataFrame(outdata)
    #print(df)
    boolean =  (df['chi2'] == np.min(df['chi2']))
    print ('\n', 'h =', list(df.loc[boolean, 'h'])[0] ,  '\n' )
    print ('Omega_m =', list(df.loc[boolean, 'Om'])[0] ,  '\n' )
    print ('Omega_r =', list(df.loc[boolean, 'Or'])[0] ,  '\n' )
    print ('chi2 =', list(df.loc[boolean, 'chi2'])[0] ,  '\n' )
    print ('reduces_chi2 =', list(df.loc[boolean, 'chi2'])[0] / (len(data) - 1) ,  '\n' )
  

In [9]:
import numpy as np
data = np.loadtxt("Legacy.dat",  dtype={'names': ('z', 'm-M' ,'sigma'), 'formats': ('f4', 'f4', 'f4')})
PrettyPrint(data)


+-------+--------+-------+
|   z   |  m-M   | sigma |
+-------+--------+-------+
|  0.05 | 36.632 | 0.045 |
| 0.031 | 35.532 | 0.091 |
| 0.075 | 37.642 | 0.049 |
| 0.026 | 35.353 | 0.094 |
| 0.101 | 38.437 | 0.055 |
|  0.02 | 34.494 | 0.111 |
| 0.045 | 36.728 | 0.057 |
| 0.043 | 36.276 | 0.059 |
| 0.018 | 34.576 | 0.121 |
| 0.079 | 37.465 | 0.041 |
| 0.088 | 38.121 | 0.046 |
| 0.063 | 37.54  | 0.046 |
| 0.026 | 35.565 | 0.141 |
|  0.05 | 36.827 |  0.06 |
| 0.071 | 37.604 | 0.048 |
| 0.025 | 35.192 | 0.092 |
| 0.053 | 36.794 | 0.047 |
| 0.024 | 35.228 | 0.094 |
| 0.016 | 34.071 | 0.146 |
| 0.049 | 36.383 | 0.051 |
| 0.016 | 34.083 | 0.138 |
| 0.125 | 38.885 | 0.049 |
| 0.035 | 35.837 | 0.069 |
| 0.016 | 34.405 | 0.133 |
| 0.017 | 34.319 | 0.133 |
|  0.03 | 35.822 |  0.21 |
|  0.03 | 35.994 |  0.08 |
| 0.017 | 34.452 | 0.136 |
| 0.028 | 35.15  | 0.079 |
| 0.054 | 36.606 | 0.054 |
| 0.024 | 35.25  | 0.102 |
| 0.017 | 34.216 | 0.128 |
| 0.039 | 36.284 | 0.057 |
| 0.032 | 35.789 | 0.074 |
|

In [10]:
runhOm(50, 0.5,  1, 100,  0, 1, data)

h = 0.7 

Omega_m = 0.26 

Omega_r = 8.4e-05 

chi2 = 113.62481604325289 

reducedchi2 = 1.0055293455155123 



In [11]:
runOm(100,0, 1,  data, 0.73 )

h = 0.73 

Omega_m = 0.17 

Omega_r = 8.4e-05 

chi2 = 127.11289623365542 

reduced_chi2 = 1.1150254055583808 



In [12]:
runh(100,0.5, 1,  data, 0.26 )


 h = 0.7 

Omega_m = 0.26 

Omega_r = 8.4e-05 

chi2 = 113.62481604325289 

reduces_chi2 = 0.9967089126601131 

